In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import seaborn as sns

module_path = (Path().resolve().parent/ "Modules")
sys.path.append(str(module_path))

pd.set_option("display.max_columns", None)
plt.rcParams["font.family"] = "IPAexGothic"

import my_modules, model_tuner, features # 自作モジュール

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
df = pd.read_csv("../Data/train_data_JV_2008.csv", encoding="cp932")

In [3]:
df = my_modules.preprocessing(df)
df = my_modules.common_process(df)

C:\Users\ken05\Documents\others\HORSE_RACING\Modules\my_modules.py:77: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["place_num"] = df["place"].replace(place_dict).astype(int)


## TrueSkill moduleの試し打ち

In [7]:
df_train = df[(df.year >= 2020) & (df.year != 2025)]
df_test = df[df.year == 2025]

display(df_train.head(3))
display(df_test.head(3))

,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,trainer_id,owner,breeding_farm,breeding_place,horse_N,waku_num,horse_num,class_code,track_code,corner_num,dist,state,weather,age_code,sex,age,basis_weight,blinker,weight,inc_dec,weight_code,win_odds,win_mul_odds_Hi,win_mul_odds_Lo,rank,time_diff,time,corner1_rank,corner2_rank,corner3_rank,corner4_rank,last_3F_time,last_3F_rank,Ave_3F,PCI,PCI3,RPCI,last_3F_time_diff,leg,pop,prize,error_code,father,mother,broodmare_sire,broodmare_sire_type,horse_color,id,id_for_fold,field_type,flat_or_jump,turn_type,race_type,waku,datetime,age_type,dist_type,is_1st_rank,is_in_3rd_rank,target
254531,202001050601010108,2020,1,5,1,中山,1,1,イズジョーボヌール,1129,1043,泉一郎,森政巳,新ひだか町,16,4,8,7,24,2,1200,良,晴,12,牝,3,54.0,NaN,450.0,2.0,3,561.3,252.4,64.6,15,2.3,1156.0,NaN,NaN,12.0,13.0,40.3,12,35.3,37.6,40.57,36.0,1.4,後方,16.0,0,0,アサクサキングス,ビバロングライフ,コマンダーインチーフ,ニアークティック系,黒鹿,2017102716,2020010506010101,ダート,平地,R,中山ダート1200,inner,2020-01-05 01:02:00,3,splint,0,0,0
254530,202001050601010116,2020,1,5,1,中山,1,1,シラカワカツコ,1173,1031,大久保與志雄,新冠橋本牧場,新冠町,16,8,16,7,24,2,1200,良,晴,12,牝,3,51.0,NaN,404.0,4.0,3,233.1,83.6,21.6,14,2.1,1154.0,NaN,NaN,9.0,10.0,40.4,13,35.0,36.6,40.57,36.0,1.1,中団,13.0,0,0,ディープブリランテ,シャイニングピサ,Smart Strike,ネイティヴダンサー系,鹿毛,2017101747,2020010506010101,ダート,平地,R,中山ダート1200,outer,2020-01-05 01:02:00,3,splint,0,0,0
254529,202001050601010115,2020,1,5,1,中山,1,1,ラブカワールド,1177,1131,増田陽一,奥山博,日高町,16,8,15,7,24,2,1200,良,晴,12,牝,3,51.0,NaN,436.0,4.0,3,474.2,153.0,39.3,13,2.0,1153.0,NaN,NaN,15.0,16.0,39.9,9,35.4,38.7,40.57,36.0,1.5,後方,15.0,0,0,トゥザワールド,ラブカテリーナ,オレハマッテルゼ,ロイヤルチャージャー系,黒鹿,2017104350,2020010506010101,ダート,平地,R,中山ダート1200,outer,2020-01-05 01:02:00,3,splint,0,0,0


,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,trainer_id,owner,breeding_farm,breeding_place,horse_N,waku_num,horse_num,class_code,track_code,corner_num,dist,state,weather,age_code,sex,age,basis_weight,blinker,weight,inc_dec,weight_code,win_odds,win_mul_odds_Hi,win_mul_odds_Lo,rank,time_diff,time,corner1_rank,corner2_rank,corner3_rank,corner4_rank,last_3F_time,last_3F_rank,Ave_3F,PCI,PCI3,RPCI,last_3F_time_diff,leg,pop,prize,error_code,father,mother,broodmare_sire,broodmare_sire_type,horse_color,id,id_for_fold,field_type,flat_or_jump,turn_type,race_type,waku,datetime,age_type,dist_type,is_1st_rank,is_in_3rd_rank,target
23662,202501050601010107,2025,1,5,1,中山,1,1,コナベイション,1170,1023,ゴドルフィン,ダーレー・ジャパン・ファーム,日高町,16,4,7,7,24,2,1200,良,晴,12,牡,3,57.0,NaN,460.0,-4.0,3,11.6,3.5,1.9,9,1.6,1138.0,NaN,NaN,3.0,3.0,39.0,14,34.8,39.2,42.5,41.5,0.3,先行,5.0,0,0,タワーオブロンドン,ボーダレス,アドマイヤムーン,ネイティヴダンサー系,鹿毛,2022101692,2025010506010101,ダート,平地,R,中山ダート1200,inner,2025-01-05 01:02:00,3,splint,0,0,0
23667,202501050601010114,2025,1,5,1,中山,1,1,ノアファースト,1169,1076,佐山公男,清水牧場,平取町,16,7,14,7,24,2,1200,良,晴,12,牡,3,57.0,B,454.0,-10.0,3,189.5,39.2,18.9,14,2.8,1150.0,NaN,NaN,16.0,14.0,38.0,5,37.0,47.4,42.5,41.5,2.5,後方,12.0,0,0,カレンブラックヒル,ノアブリジット,サマーバード,ネイティヴダンサー系,栗毛,2022102350,2025010506010101,ダート,平地,R,中山ダート1200,outer,2025-01-05 01:02:00,3,splint,0,0,0
23666,202501050601010116,2025,1,5,1,中山,1,1,カツノトゥウェルヴ,1085,1031,勝野産業,天羽牧場,日高町,16,8,16,7,24,2,1200,良,晴,12,牝,3,55.0,NaN,470.0,-10.0,3,73.0,17.3,8.5,13,2.8,1150.0,NaN,NaN,14.0,14.0,38.3,11,36.7,45.8,42.5,41.5,2.2,後方,8.0,0,0,ヘニーヒューズ,ランスタン,ダイワメジャー,ロイヤルチャージャー系,栗毛,2022101055,2025010506010101,ダート,平地,R,中山ダート1200,outer,2025-01-05 01:02:00,3,splint,0,0,0


In [9]:
from skill_calculators import trueskill_calclator

horse_ts_calculator = trueskill_calclator(target_col="horse", prefix="horse")
df_train, feature_col = horse_ts_calculator.fit_transform(df_train, [])
display(df_train[["horse_TrueSkill", "horse_TrueSkill_min", "horse_TrueSkill_sigma"]].head())
print(feature_col)

,horse_TrueSkill,horse_TrueSkill_min,horse_TrueSkill_sigma
254531,25.0,0.0,8.333333
254532,25.0,0.0,8.333333
254524,25.0,0.0,8.333333
254517,25.0,0.0,8.333333
254519,25.0,0.0,8.333333


['horse_TrueSkill', 'horse_TrueSkill_sigma', 'horse_TrueSkill_min', 'horse_TrueSkill_max']


In [10]:
df_train[df_train.horse == "イクイノックス"]

,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,trainer_id,owner,breeding_farm,breeding_place,horse_N,waku_num,horse_num,class_code,track_code,corner_num,dist,state,weather,age_code,sex,age,basis_weight,blinker,weight,inc_dec,weight_code,win_odds,win_mul_odds_Hi,win_mul_odds_Lo,rank,time_diff,time,corner1_rank,corner2_rank,corner3_rank,corner4_rank,last_3F_time,last_3F_rank,Ave_3F,PCI,PCI3,RPCI,last_3F_time_diff,leg,pop,prize,error_code,father,mother,broodmare_sire,broodmare_sire_type,horse_color,id,id_for_fold,field_type,flat_or_jump,turn_type,race_type,waku,datetime,age_type,dist_type,is_1st_rank,is_in_3rd_rank,target,horse_TrueSkill,horse_TrueSkill_sigma,horse_TrueSkill_min,horse_TrueSkill_max,horse_TrueSkill_after_racing
177449,202108280404050502,2021,8,28,4,新潟,5,5,イクイノックス,5339,1051,シルクレーシング,ノーザンファーム,安平町,15,2,2,15,12,2,1800,良,曇,11,牡,2,54.0,NaN,474.0,NaN,3,4.6,2.1,1.4,1,-1.0,1474.0,NaN,NaN,3.0,3.0,34.5,1,36.45,55.7,54.53,54.8,0.2,先行,2.0,700,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2021082804040505,芝,平地,L,新潟芝1800,inner,2021-08-28 05:06:00,2,mile,1,1,1,25.000000,8.333333,0.000000,50.000000,37.943884
166999,202111200505051101,2021,11,20,5,東京,5,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,12,1,1,179,11,2,1800,良,晴,11,牡,2,55.0,NaN,482.0,8.0,3,2.6,1.6,1.3,1,-0.4,1462.0,NaN,9.0,10.0,8.0,32.9,1,36.65,61.4,59.30,54.8,1.4,中団,1.0,3800,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2021112005050511,芝,平地,L,東京芝1800,inner,2021-11-20 11:01:00,2,mile,1,1,1,37.943884,5.522711,21.375752,54.512017,44.061739
147780,202204170603081118,2022,4,17,3,中山,8,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,18,8,18,195,17,4,2000,良,曇,12,牡,3,57.0,NaN,492.0,10.0,4,5.7,2.8,2.0,2,0.1,1598.0,7.0,5.0,4.0,3.0,34.6,8,36.51,55.5,57.17,54.1,0.4,先行,3.0,6000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2022041706030811,芝,平地,R,中山芝2000,outer,2022-04-17 11:02:00,3,intermediate,0,1,2,44.061739,4.152649,31.603792,56.519686,46.868687
142136,202205290502121118,2022,5,29,2,東京,C,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,18,8,18,195,11,4,2400,良,晴,12,牡,3,57.0,NaN,484.0,-8.0,4,3.8,1.8,1.3,2,0.0,2219.0,16.0,16.0,16.0,14.0,33.6,1,36.10,57.4,55.10,51.0,1.6,後方,2.0,8000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2022052905021211,芝,平地,L,東京芝2400,outer,2022-05-29 11:01:00,3,long,0,1,2,46.868687,3.176717,37.338537,56.398837,48.122281
123582,202210300504091107,2022,10,30,4,東京,9,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,15,4,7,195,11,3,2000,良,晴,13,牡,3,56.0,NaN,488.0,4.0,4,2.6,1.4,1.1,1,-0.1,1575.0,NaN,10.0,10.0,9.0,32.7,1,36.34,61.1,55.37,44.4,4.0,中団,1.0,20000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2022103005040911,芝,平地,L,東京芝2000,inner,2022-10-30 11:01:00,3,intermediate,1,1,1,48.122281,2.672780,40.103942,56.140620,49.067328
116232,202212250605081109,2022,12,25,5,中山,8,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,16,5,9,195,17,5,2500,良,晴,13,牡,3,55.0,NaN,492.0,4.0,4,2.3,1.4,1.1,1,-0.4,2324.0,8.0,9.0,6.0,3.0,35.4,2,36.95,54.4,54.57,52.5,0.5,先行,1.0,40000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2022122506050811,芝,平地,R,中山芝2500,outer,2022-12-25 11:02:00,3,long,1,1,1,49.067328,2.468353,41.662268,56.472388,49.748247
92628,202306250903081105,2023,6,25,3,阪神,8,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,17,3,5,195,17,4,2200,良,曇,13,牡,4,58.0,NaN,492.0,NaN,4,1.3,1.1,1.1,1,-0.0,2112.0,16.0,16.0,13.0,9.0,34.8,2,36.15,53.9,53.83,51.1,0.7,中団,1.0,22000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2023062509030811,芝,平地,R,阪神芝2200,inner,2023-06-25 11:03:00,over4,long,1,1,1,49.748247,2.318893,42.791569,56.704924,50.379890
77341,202310290504091107,2023,10,29,4,東京,9,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,11,6,7,195,11,3,2000,良,晴,13,牡,4,58.0,NaN,494.0,2.0,4,1.3,1.1,1.1,1,-0.4,1552.0,NaN,3.0,3.0,3.0,34.2,3,34.71,51.5,53.07,49.4,0.5,先行,1.0,22000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2023102905040911,芝,平地,L,東京芝2000,outer,2023-10-29 11:01:00,over4,intermediate,1,1,1,50.379890,2.192047,43.803748,56.956031,51.069069
73604,20

In [11]:
df_test_calced, feature_col_pred = horse_ts_calculator.transform(df_test, [])

In [14]:
df_test_calced[df_test.horse == "クロワデュノール"]

,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,trainer_id,owner,breeding_farm,breeding_place,horse_N,waku_num,horse_num,class_code,track_code,corner_num,dist,state,weather,age_code,sex,age,basis_weight,blinker,weight,inc_dec,weight_code,win_odds,win_mul_odds_Hi,win_mul_odds_Lo,rank,time_diff,time,corner1_rank,corner2_rank,corner3_rank,corner4_rank,last_3F_time,last_3F_rank,Ave_3F,PCI,PCI3,RPCI,last_3F_time_diff,leg,pop,prize,error_code,father,mother,broodmare_sire,broodmare_sire_type,horse_color,id,id_for_fold,field_type,flat_or_jump,turn_type,race_type,waku,datetime,age_type,dist_type,is_1st_rank,is_in_3rd_rank,target,horse_TrueSkill,horse_TrueSkill_sigma,horse_TrueSkill_min,horse_TrueSkill_max
9264,202504200603081110,2025,4,20,3,中山,8,11,クロワデュノール,1102,1151,サンデーレーシング,ノーザンファーム,安平町,18,5,10,195,17,4,2000,良,曇,12,牡,3,57.0,NaN,500.0,4.0,3,1.5,1.1,1.1,2,0.3,1573.0,4.0,4.0,6.0,2.0,34.7,8,35.4,52.0,53.87,51.2,0.4,先行,1.0,8000,0,キタサンブラック,ライジングクロス,Cape Cross,ニアークティック系,青鹿,2022105102,2025042006030811,芝,平地,R,中山芝2000,outer,2025-04-20 11:02:00,3,intermediate,0,1,2,46.571922,3.610749,35.739674,57.40417
3583,202506010502121113,2025,6,1,2,東京,C,11,クロワデュノール,1102,1151,サンデーレーシング,ノーザンファーム,安平町,18,7,13,195,11,4,2400,良,晴,12,牡,3,57.0,NaN,504.0,4.0,3,2.1,1.2,1.1,1,-0.1,2237.0,4.0,3.0,2.0,3.0,34.2,4,36.5,56.7,57.40,54.3,0.6,先行,1.0,30000,0,キタサンブラック,ライジングクロス,Cape Cross,ニアークティック系,青鹿,2022105102,2025060105021211,芝,平地,L,東京芝2400,outer,2025-06-01 11:01:00,3,long,1,1,1,46.571922,3.610749,35.739674,57.40417


In [15]:
print(feature_col_pred)

['horse_TrueSkill', 'horse_TrueSkill_sigma', 'horse_TrueSkill_min', 'horse_TrueSkill_max']


# 大丈夫そう